In [42]:
import csv
import math
import itertools
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from subprocess import check_output
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import mean_squared_error

print(check_output(["ls", "/Users/Hermione/MasterUCL/Web economics/Assignment/dataset"]).decode("utf8"))

cleantest.csv
cleantrain.csv
cleanvalidation.csv
test.csv
train.csv
validation.csv



# 1. Feature Selection
Using Gradient Boosting Decision Tree to select important features. 

The input of the GBDT is the one-hot encoded features generated from raw data using DataPreprocessing.py. 

For each input x, it has 584 features, the output of GBDT selected 39 important features.


In [34]:
train = pd.read_csv("/Users/Hermione/MasterUCL/Web economics/Assignment/dataset/train.csv")
validation = pd.read_csv("/Users/Hermione/MasterUCL/Web economics/Assignment/dataset/validation.csv")

In [4]:
ytrain = pd.read_csv("/Users/Hermione/MasterUCL/Web economics/Assignment/dataset/train.csv")["click"]
yvalidation = pd.read_csv("/Users/Hermione/MasterUCL/Web economics/Assignment/dataset/validation.csv")["click"]

# this correct the missing columns in validation and test data set due to I encoded them seperately
def MissingColumnsCorrector(df1,df2):
    for columns in df1:
        if columns in df2:
            continue
        else:
            missing_columns = columns
            ind = df1.columns.get_loc(missing_columns)
            df2.insert(ind,missing_columns,0.0)
    return df2


xtrain = pd.read_csv("/Users/Hermione/MasterUCL/Web economics/Assignment/dataset/cleantrain.csv")
#xtest = pd.read_csv("/Users/Hermione/MasterUCL/Web economics/Assignment/dataset/cleantest.csv")
xvalidation = pd.read_csv("/Users/Hermione/MasterUCL/Web economics/Assignment/dataset/cleanvalidation.csv")

xvalidation = MissingColumnsCorrector(xtrain,xvalidation)

xtrain = np.array(xtrain)
ytrian = np.array(ytrain)
ytrain = [int(numeric_string) for numeric_string in ytrain]
xvalidation = np.array(xvalidation)
yvalidation = np.array(yvalidation)
yvalidation = [int(numeric_string) for numeric_string in yvalidation]

In [ ]:
#using GBDT to find the feature importance
gbdt = GradientBoostingClassifier()
gbdt.fit(xtrain, ytrain)
feature_importance = gbdt.feature_importances_
print(feature_importance)
print("The total feature number is:",gbdt.feature_importances_.shape)


In [ ]:
#get the index of important features, and save as csv for safety 
index = np.where(feature_importance > 0)
index = list(index[0])
print("The number of important feature is:"len(index))


with open('importantFeatureInd', 'w') as myfile:
    wr = csv.writer(myfile,  dialect='excel')
    wr.writerow(index)

In [6]:
index = [0,169,197,309,355,382,400,441,443,447,450,455,458,461,462,466,467,484,486,488,500,501,503,505,506,512,523,530,542,549,555,559,561,564,565,568,569,580,582]
print("The number of important feature is:",len(index))

The number of important feature is: 39


In [7]:
#generate new train data and validation data with only important features
new_train = xtrain[:, index]
new_val = xvalidation[:, index]

# 2. Train logistic regression model
2.1 Train logistic regression model with 584 encoded features and test it on validation set

In [20]:
#train classifier
model = LogisticRegression(class_weight = "balanced")
trainedlr = model.fit(xtrain,ytrain)


In [21]:
prob = trainedlr.predict_proba(xvalidation)

pClick = pd.DataFrame(prob)
predicty = trainedlr.predict(xvalidation)
precision = precision_score(yvalidation, predicty, average='micro')
correctpred = sum(predicty == yvalidation)


In [173]:
temp = len(train) / (2 * np.bincount(train.click))
w = temp[0]/temp[1]
print(w)

LRprob =[]
for p in pClick[1]:
    LRprob.append( p / (p + ((1-p)/w)))
LRprob[:5]

0.000754533880574


[0.00054017400040291374,
 0.0042812693741423437,
 0.0010873878524688657,
 0.00085829272510502819,
 0.00046335760608389213]

In [22]:
print("lr model predicted",correctpred,"correct click status")
print("The precision of lr model is",precision)
print(pClick)

lr model predicted 225313 correct click status
The precision of lr model is 0.751672232434
               0         1
0       0.582652  0.417348
1       0.149288  0.850712
2       0.409382  0.590618
3       0.467619  0.532381
4       0.619432  0.380568
5       0.479779  0.520221
6       0.438161  0.561839
7       0.552823  0.447177
8       0.601464  0.398536
9       0.474166  0.525834
10      0.448029  0.551971
11      0.323780  0.676220
12      0.555458  0.444542
13      0.258690  0.741310
14      0.556464  0.443536
15      0.701694  0.298306
16      0.474980  0.525020
17      0.516823  0.483177
18      0.548258  0.451742
19      0.562471  0.437529
20      0.648080  0.351920
21      0.630845  0.369155
22      0.654046  0.345954
23      0.661568  0.338432
24      0.568472  0.431528
25      0.601418  0.398582
26      0.690712  0.309288
27      0.554625  0.445375
28      0.635074  0.364926
29      0.631475  0.368525
...          ...       ...
299719  0.710449  0.289551
299720  0.535661  

 
2.2 Train logistic regression model with important features selected by GBDT

In [13]:
#train classifier
lr = LogisticRegression(class_weight = "balanced")
gbdtlr = lr.fit(new_train, ytrain)


In [23]:
gbdt_predict_labels = gbdtlr.predict(new_val)
gbdtprob = gbdtlr.predict_proba(new_val)

gbdtpClick = pd.DataFrame(gbdtprob)

gbdtprecision = precision_score(yvalidation, gbdt_predict_labels, average='micro')
gbdtcorrectpred = sum(gbdt_predict_labels == yvalidation)

In [164]:
print("GBDTlr model predicted",gbdtcorrectpred,"correct click status")
print("lr model predicted",correctpred,"correct click status")


print("The precision of GBDTlr model is",gbdtprecision)
print("The precision of lr model is",precision)
print(gbdtpClick)

GBDTlr model predicted 247662 correct click status
lr model predicted 225313 correct click status
The precision of GBDTlr model is 0.826231280171
The precision of lr model is 0.751672232434
               0         1
0       0.774397  0.225603
1       0.093838  0.906162
2       0.489235  0.510765
3       0.544563  0.455437
4       0.607295  0.392705
5       0.551713  0.448287
6       0.489235  0.510765
7       0.576059  0.423941
8       0.784149  0.215851
9       0.462148  0.537852
10      0.574666  0.425334
11      0.379136  0.620864
12      0.705581  0.294419
13      0.256016  0.743984
14      0.695653  0.304347
15      0.774397  0.225603
16      0.607295  0.392705
17      0.492623  0.507377
18      0.655424  0.344576
19      0.603637  0.396363
20      0.607295  0.392705
21      0.607295  0.392705
22      0.774686  0.225314
23      0.837479  0.162521
24      0.720936  0.279064
25      0.627780  0.372220
26      0.708751  0.291249
27      0.624195  0.375805
28      0.562945  0.437055


In [32]:
RMSEgbdt = mean_squared_error(yvalidation, gbdt_predict_labels) 

0.17376871982892353

In [35]:
# apply negative downsampling to work out weights in order for probability of click to have the same ratio as training data, technique is called 
#model recalibration  
temp = len(train) / (2 * np.bincount(train.click))
w = temp[0]/temp[1]
print(w)

GBDTprob =[]
for p in gbdtpClick[1]:
    GBDTprob.append( p / (p + ((1-p)/w)))
GBDTprob[:5]

0.000754533880574


[0.00021976783228367061,
 0.0072335538424793998,
 0.00078711926378626645,
 0.00063064617239325289,
 0.00048767895148636459]

In [128]:
p = 0.3
pro = p / (p + ((1-p)/w))

In [129]:
pro

0.00032326712767462225

In [147]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve([click for click in validation.click], GBDTprob)
print('AUC accuracy of GBDTlr:',metrics.auc(fpr, tpr))

AUC accuracy of GBDTlr: 0.716542045215


In [38]:
RMSEgbdt = mean_squared_error(yvalidation, GBDTprob) 
RMSEgbdt

0.00075199772192006053

# 3. Optimal Real-Time Bidding(ORTB)
In this section, two ORTB solutions will be used to find the best one.

In [105]:
lambdas = [5.2e-10,5.2e-9,5.2e-8,5.2e-7,5.2e-6,5.2e-5,5.2e-4,5.2e-3,5.2e-2,5.2e-1]
c = np.arange(10,101,10).tolist()
combinations = list(itertools.product(c, lambdas))

In [98]:
c

[10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

In [99]:
lambdas

[5.2e-10,
 5.2e-09,
 5.2e-08,
 5.2e-07,
 5.2e-06,
 5.2e-05,
 0.00052,
 0.0052,
 0.052,
 0.52]

In [ ]:
#c=2
#lambdas = 1
#ctr = 2
#ortb1 = math.sqrt(c/lambdas*ctr + c**2)-c
#ortb2 = c*(math.pow((ctr+math.sqrt((c**2)*(lambdas**2)+ctr**2))/(c*lambdas),1/3)-math.pow(c*lambdas/(ctr+math.sqrt((c**2)*(lambdas**2)+ctr**2)),1/3))

In [55]:
GBDTctr = np.asarray(GBDTprob)

In [151]:
def ortb1(c,lambdas,predictctr):
    impression = 0.0
    clicks = 0
    cost = 0.0
    budget = 2500000  # 1/10th budget of 25,000,000
    bidortb1 = []
    
    for pctr in predictctr:
        ortb1 = math.sqrt(c/lambdas*pctr + c**2)-c
        bidortb1.append(ortb1)
    
    true_bids = bidortb1 >= validation.payprice
    for i in range(0, len(true_bids)):
        if true_bids[i] == True:
            impression += 1.0
            clicks += validation.click[i]
            cost += validation.payprice[i]
        if cost >= budget:
            break
    return impression, clicks, cost
    
    

In [152]:
def ortb2(c,lambdas,predictctr):
    impression = 0.0
    clicks = 0
    cost = 0.0
    budget = 2500000  # 1/10th budget of 25,000,000
    bidortb2 = []
    
    for pctr in predictctr:
        ortb2 = c*(math.pow((pctr+math.sqrt((c**2)*(lambdas**2)+pctr**2))/(c*lambdas),1/3)-math.pow(c*lambdas/(pctr+math.sqrt((c**2)*(lambdas**2)+pctr**2)),1/3))
        bidortb2.append(ortb2)
    
    true_bids = bidortb2 >= validation.payprice
    for i in range(0, len(true_bids)):
        if true_bids[i] == True:
            impression += 1.0
            clicks += validation.click[i]
            cost += validation.payprice[i]
        if cost >= budget:
            break
    return impression, clicks, cost

In [153]:
def get_results(predictctr,ortb):
    df = pd.DataFrame()
    
    imp = []
    clik = []
    ct = []
    clamb = []
    for combination in combinations:
        c = combination[0]
        lam = combination[1]
        [imps, clicks, cost] = ortb(c,lam,predictctr)
        imp.append(imps)
        clik.append(clicks)
        ct.append(cost)
        clamb.append(combination)
    df['C_Lambda'] = clamb
    df['impressions'] = imp
    df['total_cost'] = ct
    df['clicks'] = clik
    df['CTR'] = (df.clicks / df.impressions * 100).round(2).astype(str)
    df['CPM'] = (df.total_cost / df.impressions * 1000).round(2).astype(str)
    df['CPC'] = (df.total_cost / df.clicks).round(2).astype(str)
    return df


In [154]:
ortb1result = get_results(GBDTctr,ortb1)
       

In [155]:
ortb2result = get_results(GBDTctr,ortb2)

In [156]:
ortb1result.sort_values("CTR",ascending= False).head(10)

,C_Lambda,impressions,total_cost,clicks,CTR,CPM,CPC
25,"(30, 5.2e-05)",5275.0,51435.0,12,0.23,9750.71,4286.25
95,"(100, 5.2e-05)",6415.0,84724.0,14,0.22,13207.17,6051.71
45,"(50, 5.2e-05)",5902.0,67722.0,13,0.22,11474.42,5209.38
55,"(60, 5.2e-05)",6021.0,70835.0,13,0.22,11764.66,5448.85
75,"(80, 5.2e-05)",6274.0,79666.0,13,0.21,12697.8,6128.15
65,"(70, 5.2e-05)",6165.0,75652.0,13,0.21,12271.21,5819.38
35,"(40, 5.2e-05)",5753.0,63274.0,12,0.21,10998.44,5272.83
5,"(10, 5.2e-05)",3936.0,29100.0,8,0.2,7393.29,3637.5
85,"(90, 5.2e-05)",6372.0,83245.0,13,0.2,13064.19,6403.46
15,"(20, 5.2e-05)",4898.0,42267.0,9,0.18,8629.44,4696.33


In [157]:
ortb2result.sort_values("CTR",ascending= False).head(10)

,C_Lambda,impressions,total_cost,clicks,CTR,CPM,CPC
65,"(70, 5.2e-05)",10219.0,154349.0,16,0.16,15104.12,9646.81
35,"(40, 5.2e-05)",9417.0,119381.0,15,0.16,12677.18,7958.73
85,"(90, 5.2e-05)",10412.0,164902.0,16,0.15,15837.69,10306.38
95,"(100, 5.2e-05)",10522.0,169801.0,16,0.15,16137.71,10612.56
55,"(60, 5.2e-05)",10044.0,146165.0,15,0.15,14552.47,9744.33
75,"(80, 5.2e-05)",10341.0,160771.0,16,0.15,15546.95,10048.19
45,"(50, 5.2e-05)",9782.0,134071.0,15,0.15,13705.89,8938.07
15,"(20, 5.2e-05)",8076.0,77454.0,11,0.14,9590.64,7041.27
25,"(30, 5.2e-05)",8794.0,97546.0,12,0.14,11092.34,8128.83
5,"(10, 5.2e-05)",6310.0,43279.0,8,0.13,6858.8,5409.88


In [158]:
ortb1result.sort_values("clicks",ascending= False).head(10)

,C_Lambda,impressions,total_cost,clicks,CTR,CPM,CPC
94,"(100, 5.2e-06)",77192.0,2410834.0,95,0.12,31231.66,25377.2
84,"(90, 5.2e-06)",75920.0,2325259.0,95,0.13,30627.75,24476.41
74,"(80, 5.2e-06)",74506.0,2238576.0,92,0.12,30045.58,24332.35
64,"(70, 5.2e-06)",72310.0,2108819.0,89,0.12,29163.59,23694.6
54,"(60, 5.2e-06)",70894.0,2027379.0,86,0.12,28597.33,23574.17
44,"(50, 5.2e-06)",68274.0,1884086.0,78,0.11,27595.95,24154.95
34,"(40, 5.2e-06)",64150.0,1662688.0,74,0.12,25918.75,22468.76
24,"(30, 5.2e-06)",59657.0,1448401.0,63,0.11,24278.81,22990.49
3,"(10, 5.2e-07)",55209.0,2500003.0,56,0.1,45282.53,44642.91
14,"(20, 5.2e-06)",51677.0,1115103.0,49,0.09,21578.32,22757.2


In [159]:
ortb2result.sort_values("clicks",ascending= False).head(10)

,C_Lambda,impressions,total_cost,clicks,CTR,CPM,CPC
34,"(40, 5.2e-06)",85773.0,2349603.0,73,0.09,27393.27,32186.34
54,"(60, 5.2e-06)",77252.0,2500015.0,72,0.09,32361.82,34722.43
64,"(70, 5.2e-06)",73685.0,2500002.0,71,0.1,33928.24,35211.3
44,"(50, 5.2e-06)",83397.0,2500005.0,71,0.09,29977.16,35211.34
84,"(90, 5.2e-06)",69322.0,2500034.0,70,0.1,36064.08,35714.77
74,"(80, 5.2e-06)",71209.0,2500046.0,69,0.1,35108.57,36232.55
94,"(100, 5.2e-06)",68035.0,2500041.0,68,0.1,36746.4,36765.31
24,"(30, 5.2e-06)",76329.0,1847168.0,62,0.08,24200.08,29793.03
3,"(10, 5.2e-07)",82844.0,2500006.0,62,0.07,30177.27,40322.68
13,"(20, 5.2e-07)",56567.0,2500053.0,52,0.09,44196.32,48077.94


In [160]:
def ortb1opt(c,lambdas,predictctr):
    impression = 0.0
    clicks = 0
    cost = 0.0
    budget = 2500000  # 1/10th budget of 25,000,000
    bidortb1 = []
    
    for pctr in predictctr:
        if pctr <= 0.0003:
            ortb1 = 0
        else:
            ortb1 = math.sqrt(c/lambdas*pctr + c**2)-c
        bidortb1.append(ortb1)
    
    true_bids = bidortb1 >= validation.payprice
    for i in range(0, len(true_bids)):
        if true_bids[i] == True:
            impression += 1.0
            clicks += validation.click[i]
            cost += validation.payprice[i]
        if cost >= budget:
            break
    return impression, clicks, cost
    
    

In [161]:
ortb1opt = get_results(GBDTctr,ortb1opt)

In [167]:
def ortb2opt(c,lambdas,predictctr):
    impression = 0.0
    clicks = 0
    cost = 0.0
    budget = 2500000  # 1/10th budget of 25,000,000
    bidortb2 = []
    
    for pctr in predictctr:
        if pctr <= 0.0003:
            bidprice = 0
        else:
            bidprice = c*(math.pow((pctr+math.sqrt((c**2)*(lambdas**2)+pctr**2))/(c*lambdas),1/3)-math.pow(c*lambdas/(pctr+math.sqrt((c**2)*(lambdas**2)+pctr**2)),1/3))
        bidortb2.append(bidprice)
    
    true_bids = bidortb2 >= validation.payprice
    for i in range(0, len(true_bids)):
        if true_bids[i] == True:
            impression += 1.0
            clicks += validation.click[i]
            cost += validation.payprice[i]
        if cost >= budget:
            break
    return impression, clicks, cost

In [168]:
ortb2opt = get_results(GBDTctr,ortb2opt)

In [169]:
ortb1opt.sort_values("clicks",ascending= False).head(10)

,C_Lambda,impressions,total_cost,clicks,CTR,CPM,CPC
94,"(100, 5.2e-06)",64425.0,2259176.0,94,0.15,35066.76,24033.79
84,"(90, 5.2e-06)",63235.0,2175323.0,94,0.15,34400.62,23141.73
74,"(80, 5.2e-06)",62038.0,2092803.0,91,0.15,33734.21,22997.84
64,"(70, 5.2e-06)",60209.0,1970126.0,88,0.15,32721.45,22387.8
54,"(60, 5.2e-06)",58981.0,1892236.0,85,0.14,32082.13,22261.6
44,"(50, 5.2e-06)",56968.0,1760962.0,77,0.14,30911.42,22869.64
34,"(40, 5.2e-06)",53295.0,1547899.0,73,0.14,29043.98,21204.1
24,"(30, 5.2e-06)",49753.0,1350544.0,62,0.12,27144.98,21782.97
3,"(10, 5.2e-07)",48504.0,2500024.0,61,0.13,51542.64,40984.0
13,"(20, 5.2e-07)",42049.0,2500035.0,51,0.12,59455.28,49020.29


In [170]:
ortb2opt.sort_values("clicks",ascending= False).head(10)

,C_Lambda,impressions,total_cost,clicks,CTR,CPM,CPC
44,"(50, 5.2e-06)",72671.0,2485265.0,84,0.12,34198.86,29586.49
54,"(60, 5.2e-06)",67732.0,2500001.0,77,0.11,36910.19,32467.55
64,"(70, 5.2e-06)",64786.0,2500001.0,76,0.12,38588.6,32894.75
74,"(80, 5.2e-06)",62397.0,2500009.0,76,0.12,40066.17,32894.86
84,"(90, 5.2e-06)",60833.0,2500051.0,74,0.12,41096.95,33784.47
3,"(10, 5.2e-07)",73093.0,2500037.0,73,0.1,34203.51,34247.08
94,"(100, 5.2e-06)",59735.0,2500124.0,71,0.12,41853.59,35213.01
34,"(40, 5.2e-06)",65699.0,2046964.0,71,0.11,31156.7,28830.48
13,"(20, 5.2e-07)",50854.0,2500037.0,65,0.13,49161.07,38462.11
24,"(30, 5.2e-06)",57737.0,1578916.0,60,0.1,27346.69,26315.27


In [171]:
ortb1opt.sort_values("CTR",ascending= False).head(10)

,C_Lambda,impressions,total_cost,clicks,CTR,CPM,CPC
25,"(30, 5.2e-05)",5163.0,51272.0,12,0.23,9930.66,4272.67
55,"(60, 5.2e-05)",5909.0,70672.0,13,0.22,11960.06,5436.31
95,"(100, 5.2e-05)",6303.0,84561.0,14,0.22,13415.99,6040.07
45,"(50, 5.2e-05)",5790.0,67559.0,13,0.22,11668.22,5196.85
75,"(80, 5.2e-05)",6162.0,79503.0,13,0.21,12902.14,6115.62
5,"(10, 5.2e-05)",3836.0,28961.0,8,0.21,7549.79,3620.12
35,"(40, 5.2e-05)",5641.0,63111.0,12,0.21,11187.91,5259.25
85,"(90, 5.2e-05)",6260.0,83082.0,13,0.21,13271.88,6390.92
65,"(70, 5.2e-05)",6053.0,75489.0,13,0.21,12471.34,5806.85
15,"(20, 5.2e-05)",4786.0,42104.0,9,0.19,8797.33,4678.22


In [172]:
ortb2opt.sort_values("CTR",ascending= False).head(10)

,C_Lambda,impressions,total_cost,clicks,CTR,CPM,CPC
65,"(70, 5.2e-05)",10068.0,154105.0,16,0.16,15306.42,9631.56
45,"(50, 5.2e-05)",9631.0,133827.0,15,0.16,13895.44,8921.8
35,"(40, 5.2e-05)",9266.0,119137.0,15,0.16,12857.44,7942.47
75,"(80, 5.2e-05)",10190.0,160527.0,16,0.16,15753.39,10032.94
85,"(90, 5.2e-05)",10261.0,164658.0,16,0.16,16046.97,10291.12
95,"(100, 5.2e-05)",10371.0,169557.0,16,0.15,16349.15,10597.31
55,"(60, 5.2e-05)",9893.0,145921.0,15,0.15,14749.92,9728.07
15,"(20, 5.2e-05)",7925.0,77210.0,11,0.14,9742.59,7019.09
25,"(30, 5.2e-05)",8643.0,97302.0,12,0.14,11257.9,8108.5
62,"(70, 5.2e-08)",29795.0,2500040.0,38,0.13,83908.04,65790.53


In [177]:
LRctr = np.asarray(LRprob)

In [184]:
LRctr

array([ 0.00054017,  0.00428127,  0.00108739, ...,  0.0004593 ,
        0.00027177,  0.00063145])

In [179]:
LRortb1result = get_results(LRctr,ortb1)


In [186]:
LRortb1result = pd.DataFrame()
    
lrimp = []
lrclik = []
lrct = []
lrclamb = []
for combination in combinations:
    c = combination[0]
    lam = combination[1]
    [lrimps, lrclicks, lrcost] = ortb1(c,lam,LRprob)
    lrimp.append(lrimps)
    lrclik.append(lrclicks)
    lrct.append(lrcost)
    lrclamb.append(combination)
LRortb1result['C_Lambda'] = lrclamb
LRortb1result['impressions'] = lrimp
LRortb1result['total_cost'] = lrct
LRortb1result['clicks'] = lrclik
LRortb1result['CTR'] = (LRortb1result.clicks / LRortb1result.impressions * 100).round(2).astype(str)
LRortb1result['CPM'] = (LRortb1result.total_cost / LRortb1result.impressions * 1000).round(2).astype(str)
LRortb1result['CPC'] = (LRortb1result.total_cost / LRortb1result.clicks).round(2).astype(str)

In [180]:
LRortb2result = get_results(LRctr,ortb2)


In [192]:
LRortb1opt = get_results(LRctr,ortb1opt)


TypeError: 'DataFrame' object is not callable

In [ ]:
LRortb2opt = get_results(LRctr,ortb2opt)

In [188]:
LRortb1result.sort_values("CTR",ascending= False).head(5)

,C_Lambda,impressions,total_cost,clicks,CTR,CPM,CPC
5,"(10, 5.2e-05)",4230.0,28054.0,8,0.19,6632.15,3506.75
85,"(90, 5.2e-05)",6972.0,74224.0,13,0.19,10646.01,5709.54
45,"(50, 5.2e-05)",6504.0,63827.0,12,0.18,9813.5,5318.92
95,"(100, 5.2e-05)",7031.0,75494.0,13,0.18,10737.31,5807.23
35,"(40, 5.2e-05)",6269.0,59323.0,11,0.18,9462.91,5393.0


In [190]:
LRortb2result.sort_values("CTR",ascending= False).head(5)

,C_Lambda,impressions,total_cost,clicks,CTR,CPM,CPC
46,"(50, 0.00052)",392.0,1616.0,6,1.53,4122.45,269.33
56,"(60, 0.00052)",397.0,1646.0,6,1.51,4146.1,274.33
66,"(70, 0.00052)",398.0,1652.0,6,1.51,4150.75,275.33
86,"(90, 0.00052)",401.0,1670.0,6,1.5,4164.59,278.33
76,"(80, 0.00052)",399.0,1658.0,6,1.5,4155.39,276.33


In [ ]:
LRortb1opt.sort_values("CTR",ascending= False).head(5)

In [ ]:
LRortb2opt.sort_values("CTR",ascending= False).head(5)

In [189]:
LRortb1result.sort_values("clicks",ascending= False).head(5)

,C_Lambda,impressions,total_cost,clicks,CTR,CPM,CPC
44,"(50, 5.2e-06)",87737.0,2500023.0,82,0.09,28494.51,30488.09
84,"(90, 5.2e-06)",77144.0,2500043.0,79,0.1,32407.48,31646.11
54,"(60, 5.2e-06)",83863.0,2500018.0,77,0.09,29810.74,32467.77
94,"(100, 5.2e-06)",75733.0,2500012.0,77,0.1,33010.87,32467.69
34,"(40, 5.2e-06)",84118.0,2256850.0,76,0.09,26829.57,29695.39


In [191]:
LRortb2result.sort_values("clicks",ascending= False).head(5)

,C_Lambda,impressions,total_cost,clicks,CTR,CPM,CPC
44,"(50, 5.2e-06)",74531.0,2500049.0,71,0.1,33543.75,35211.96
34,"(40, 5.2e-06)",82224.0,2500003.0,69,0.08,30404.78,36231.93
54,"(60, 5.2e-06)",69604.0,2500011.0,64,0.09,35917.63,39062.67
24,"(30, 5.2e-06)",93705.0,2443704.0,63,0.07,26078.69,38788.95
94,"(100, 5.2e-06)",61048.0,2500013.0,63,0.1,40951.6,39682.75


In [ ]:
LRortb1opt.sort_values("clicks",ascending= False).head(5)

In [ ]:
LRortb2opt.sort_values("clicks",ascending= False).head(5)